In [29]:
import pandas as pd
import numpy as np
import tkinter as tk
import warnings
import locale
import calendar
import chardet
import os
from tkinter import messagebox
from datetime import datetime
from unidecode import unidecode

warnings.filterwarnings('ignore')
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

rj_path = 'bases/gestiones/RJ'
clasa_path = 'bases/gestiones/CLASA'
mornese_path = 'bases/gestiones/MORNESE'

rj_archivos = [file for file in os.listdir(rj_path) if file.startswith('REPORTE_CORTES_BBVA_') and file.endswith('.xlsx')]
clasa_archivos = [file for file in os.listdir(clasa_path) if file.startswith('REPORTE_CORTES_BBVA_') and file.endswith('.txt')]
mornese_archivos = [file for file in os.listdir(mornese_path) if file.startswith('REPORTE_CORTES_BBVA_') and file.endswith('.xlsx')]

print(rj_archivos)
print(clasa_archivos)
print(mornese_archivos)

['REPORTE_CORTES_BBVA_2024-09.xlsx', 'REPORTE_CORTES_BBVA_2024-10.xlsx', 'REPORTE_CORTES_BBVA_2024-11.xlsx']
['REPORTE_CORTES_BBVA_2024-09.txt', 'REPORTE_CORTES_BBVA_2024-10.txt', 'REPORTE_CORTES_BBVA_2024-11.txt']
['REPORTE_CORTES_BBVA_2024-09.xlsx', 'REPORTE_CORTES_BBVA_2024-10.xlsx', 'REPORTE_CORTES_BBVA_2024-11.xlsx']


In [30]:
def clean_columns(columns_list: list[str]) -> list[str]:
    return [column.strip().replace('.', '').replace(' ', '_').upper() for column in columns_list]

In [31]:
def detectar_codificacion(ruta_archivo):
    with open(ruta_archivo, 'rb') as f:
        resultado = chardet.detect(f.read())
    return resultado['encoding']

In [32]:
def clean_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    if 'AGENCIA' in df.columns:
        df['AGENCIA'] = df['AGENCIA'].str.upper().replace('ASESCOM RJ ABOGADOS', 'ASESCOM RJ')
    if 'TIPO_DE_MARCACIÓN' in df.columns:
        df.rename(columns={'TIPO_DE_MARCACIÓN': 'TIPO_MARCADOR'}, inplace=True)
    if 'TIPO_DE_DISCADO' in df.columns:
        df.rename(columns={'TIPO_DE_DISCADO': 'TIPO_MARCADOR'}, inplace=True)
    if 'PRIORIDAD_TELEFONO' not in df.columns:
        df['PRIORIDAD_TELEFONO'] = np.nan
    
    df['MARCA'] = df['MARCA'].str.strip().str.upper().replace(' ', '', regex=True)
    df['MARCA'] = np.where(df['MARCA'] == 'KST', 'KSTBC', df['MARCA'])
    df = df[df['MARCA'].isin(['BC', 'EXJ', 'KSTBC'])]
    df['FECHA_LLAMADA'] = pd.to_datetime(df['FECHA_LLAMADA'], errors='coerce')
    df['HORA_INICIO_LLAMADA'] = pd.to_datetime(df['HORA_INICIO_LLAMADA']).dt.hour.astype('Int64').fillna(0)
    df['TIPO_CONTACTO'] = df['TIPO_CONTACTO'].str.upper().replace('TERCERO', 'CONTACTO INDIRECTO').replace('CONTACTO EFECTIVO', 'CONTACTO DIRECTO').replace('EFECTIVO', 'CONTACTO DIRECTO')
    df['TRAMO'] = df['TRAMO'].str.strip().str.upper().replace(' ', '', regex=True).replace(['T1', 'T2', 'T3', 'T4'], 'NULL').fillna('NULL')
    df['PDP'] = df.apply(lambda x: 1 if pd.notna(x['FECHA_PDP']) else 0, axis=1)
    df['TIPO_MARCADOR'] = df['TIPO_MARCADOR'].str.strip().str.upper().replace(' ', '', regex=True).fillna('NULL')
    
    df = df[['AGENCIA', 'COD_CENTRAL', 'CONTRATO', 'MARCA', 'FECHA_LLAMADA', 'HORA_INICIO_LLAMADA', 'TIPO_CONTACTO', 'TRAMO', 'PDP', 'TIPO_MARCADOR', 'PRIORIDAD_TELEFONO']]
    return df

In [45]:
def consolidar_archivos(agencia_path: str, archivos: list) -> pd.DataFrame:
    dataframes = []
    for archivo in archivos:
        ruta_completa = os.path.join(agencia_path, archivo)
        if archivo.endswith('.txt'):
            df = pd.read_csv(
                ruta_completa, 
                sep=',', 
                dtype={'CONTRATO': str, 'COD_CENTRAL': str, 'NRO_DOC': str}, 
                encoding='ISO-8859-1'
                )
        else:
            df = pd.read_excel(
                ruta_completa, 
                dtype={'CONTRATO': str, 'COD_CENTRAL': str, 'NRO_DOC': str}
                )
        
        df.columns = clean_columns(df.columns)
        dataframes.append(df)
        print(f'{archivo} leído correctamente')
    
    df_consolidado = pd.concat(dataframes, ignore_index=True)
    df_consolidado = clean_dataframe(df_consolidado)
    
    df_consolidado.to_csv(
        os.path.join(agencia_path, f'CONSOLIDADO_GESTIONES_{agencia_path.split("/")[-1]}.csv'), 
        index=False, 
        sep='|'
        )
    return df_consolidado

In [46]:
root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

result = messagebox.askquestion('Confirmación', '¿Consolidar archivos de gestiones?', icon='warning')

if result == 'yes':
    dataframes = {}
    archivos = {
        #'RJ': (rj_path, rj_archivos),
        'CLASA': (clasa_path, clasa_archivos),
        #'MORNESE': (mornese_path, mornese_archivos)
    }
    
    for key, (path, file_list) in archivos.items():
        if file_list:
            dataframes[key] = consolidar_archivos(path, file_list)
            print(f'Consolidado {key}:', dataframes[key].shape)
        else:
            print(f'No hay archivos para consolidar en {key}.')

root.destroy()

REPORTE_CORTES_BBVA_2024-09.txt leído correctamente
REPORTE_CORTES_BBVA_2024-10.txt leído correctamente
REPORTE_CORTES_BBVA_2024-11.txt leído correctamente
Consolidado CLASA: (1112769, 11)


In [47]:
root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

result = messagebox.askquestion('Confirmación', '¿Leer consolidados?', icon='warning')

if result == 'yes':
    dataframes = {}
    archivos = {
        #'RJ': f'{rj_path}/CONSOLIDADO_GESTIONES_RJ.csv',
        'CLASA': f'{clasa_path}/CONSOLIDADO_GESTIONES_CLASA.csv',
        #'MORNESE': f'{mornese_path}/CONSOLIDADO_GESTIONES_MORNESE.csv'
    }
    
    for key, file_path in archivos.items():
        if os.path.exists(file_path):
            dataframes[key] = pd.read_csv(
                file_path, 
                sep='|', 
                dtype={'CONTRATO': str, 'COD_CENTRAL': str, 'NRO_DOC': str}, 
                encoding='ISO-8859-1'
                )
            print(f'Consolidado {key} leído:', dataframes[key].shape)
        else:
            print(f'El archivo CONSOLIDADO_GESTIONES_{key}.csv no existe.')

root.destroy()

Consolidado CLASA leído: (1112769, 11)


In [114]:
df_clasa_test: pd.DataFrame = dataframes['CLASA'].copy()
df_clasa_test.sort_values(by='FECHA_LLAMADA', inplace=True)
df_clasa_test.reset_index(inplace=True, drop=True)
print(df_clasa_test.columns)
df_clasa_test.head(5)

Index(['AGENCIA', 'COD_CENTRAL', 'CONTRATO', 'MARCA', 'FECHA_LLAMADA',
       'HORA_INICIO_LLAMADA', 'TIPO_CONTACTO', 'TRAMO', 'PDP', 'TIPO_MARCADOR',
       'PRIORIDAD_TELEFONO'],
      dtype='object')


,AGENCIA,COD_CENTRAL,CONTRATO,MARCA,FECHA_LLAMADA,HORA_INICIO_LLAMADA,TIPO_CONTACTO,TRAMO,PDP,TIPO_MARCADOR,PRIORIDAD_TELEFONO
0,CLASA,24667250,001101135002005456,KSTBC,2024-09-02,13,NO CONTACTADO,NaN,0,PROGRESIVO,0
1,CLASA,28936749,001102759600630869,BC,2024-09-02,7,NO CONTACTADO,NaN,0,PROGRESIVO,0
2,CLASA,25093233,001102819600306584,KSTBC,2024-09-02,8,NO CONTACTADO,NaN,0,PREDICTIVO,0
3,CLASA,22405464,001102145000257759,KSTBC,2024-09-02,17,CONTACTO DIRECTO,C2,0,MANUAL,0
4,CLASA,27148070,001104379601896048,KSTBC,2024-09-02,8,NO CONTACTADO,NaN,0,PROGRESIVO,0


In [115]:
df_clasa_test['PERIODO'] = pd.to_datetime(df_clasa_test['FECHA_LLAMADA']).dt.strftime('%Y%m').astype(str)

In [116]:
fechas_unicas = df_clasa_test[['PERIODO', 'FECHA_LLAMADA']].drop_duplicates().sort_values(by='FECHA_LLAMADA')
orden_fechas = fechas_unicas.groupby('PERIODO').cumcount() + 1
fechas_unicas['DIA'] = orden_fechas
df_clasa_test = df_clasa_test.merge(fechas_unicas[['PERIODO', 'FECHA_LLAMADA', 'DIA']], on=['PERIODO', 'FECHA_LLAMADA'], how='left')

df_clasa_test['INTENSITDAD_TOTAL'] = df_clasa_test.groupby(['COD_CENTRAL', 'PERIODO'])['COD_CENTRAL'].transform('count')

In [117]:
cod_central_counts = df_clasa_test.groupby('COD_CENTRAL')['PERIODO'].nunique()
cod_central_in_three_months = cod_central_counts[cod_central_counts == 3].index
cod_central_in_three_months

Index(['00099430', '00246220', '00420425', '00453765', '00526770', '00669822',
       '00844730', '01162268', '01336690', '01616510',
       ...
       '31591267', '31594561', '31596973', '31597307', '31599055', '31599587',
       '31601892', '31604477', '31619575', '31685884'],
      dtype='object', name='COD_CENTRAL', length=9209)

In [118]:
df_clasa_test[df_clasa_test['COD_CENTRAL'] == '31596973']

,AGENCIA,COD_CENTRAL,CONTRATO,MARCA,FECHA_LLAMADA,HORA_INICIO_LLAMADA,TIPO_CONTACTO,TRAMO,PDP,TIPO_MARCADOR,PRIORIDAD_TELEFONO,PERIODO,DIA,INTENSITDAD_TOTAL
40425,CLASA,31596973,001105045005269530,KSTBC,2024-09-05,15,NO CONTACTADO,C1,0,PREDICTIVO,1,202409,4,13
197361,CLASA,31596973,001105045005269530,KSTBC,2024-09-19,10,NO CONTACTADO,C1,0,PREDICTIVO,3,202409,14,13
199794,CLASA,31596973,001105045005269530,KSTBC,2024-09-19,18,NO CONTACTADO,C1,0,MANUAL,1,202409,14,13
204892,CLASA,31596973,001105045005269530,KSTBC,2024-09-19,10,NO CONTACTADO,C1,0,PREDICTIVO,3,202409,14,13
208735,CLASA,31596973,001105045005269530,KSTBC,2024-09-19,15,NO CONTACTADO,C1,0,PREDICTIVO,3,202409,14,13
211149,CLASA,31596973,001105045005269530,KSTBC,2024-09-19,10,NO CONTACTADO,C1,0,PREDICTIVO,3,202409,14,13
211312,CLASA,31596973,001105045005269530,KSTBC,2024-09-19,12,NO CONTACTADO,C1,0,PREDICTIVO,3,202409,14,13
212663,CLASA,31596973,001105045005269530,KSTBC,2024-09-19,11,NO CONTACTADO,C1,0,PREDICTIVO,3,202409,14,13
212876,CLASA,31596973,001105045005269530,KSTBC,2024-09-19,9,NO CONTACTADO,C1,0,PREDICTIVO,3,202409,14,13
216029,CLASA,31596973,001105045005269530,KSTBC,2024-09-20,15,NO CONTACTADO,C1,0,PREDICTIVO,3,202409,15,13


In [119]:
df_clasa_test.shape

(1112769, 14)

In [120]:
df_contacto_directo = df_clasa_test[df_clasa_test['TIPO_CONTACTO'] == 'CONTACTO DIRECTO']
# Agrupa por 'COD_CENTRAL' y 'PERIODO', y cuenta el número de contactos directos en cada grupo
suma_contacto_directo = df_contacto_directo.groupby(['COD_CENTRAL', 'PERIODO', 'DIA']).size().reset_index(name='DIRECTO_CALL')

df_clasa_test = df_clasa_test.merge(suma_contacto_directo, on=['COD_CENTRAL', 'PERIODO', 'DIA'], how='left')

df_clasa_test['DIRECTO_CALL'] =  df_clasa_test['DIRECTO_CALL'].fillna(0).astype(int)

In [121]:
df_clasa_test[df_clasa_test['COD_CENTRAL'] == '31596973']

,AGENCIA,COD_CENTRAL,CONTRATO,MARCA,FECHA_LLAMADA,HORA_INICIO_LLAMADA,TIPO_CONTACTO,TRAMO,PDP,TIPO_MARCADOR,PRIORIDAD_TELEFONO,PERIODO,DIA,INTENSITDAD_TOTAL,DIRECTO_CALL
40425,CLASA,31596973,001105045005269530,KSTBC,2024-09-05,15,NO CONTACTADO,C1,0,PREDICTIVO,1,202409,4,13,0
197361,CLASA,31596973,001105045005269530,KSTBC,2024-09-19,10,NO CONTACTADO,C1,0,PREDICTIVO,3,202409,14,13,0
199794,CLASA,31596973,001105045005269530,KSTBC,2024-09-19,18,NO CONTACTADO,C1,0,MANUAL,1,202409,14,13,0
204892,CLASA,31596973,001105045005269530,KSTBC,2024-09-19,10,NO CONTACTADO,C1,0,PREDICTIVO,3,202409,14,13,0
208735,CLASA,31596973,001105045005269530,KSTBC,2024-09-19,15,NO CONTACTADO,C1,0,PREDICTIVO,3,202409,14,13,0
211149,CLASA,31596973,001105045005269530,KSTBC,2024-09-19,10,NO CONTACTADO,C1,0,PREDICTIVO,3,202409,14,13,0
211312,CLASA,31596973,001105045005269530,KSTBC,2024-09-19,12,NO CONTACTADO,C1,0,PREDICTIVO,3,202409,14,13,0
212663,CLASA,31596973,001105045005269530,KSTBC,2024-09-19,11,NO CONTACTADO,C1,0,PREDICTIVO,3,202409,14,13,0
212876,CLASA,31596973,001105045005269530,KSTBC,2024-09-19,9,NO CONTACTADO,C1,0,PREDICTIVO,3,202409,14,13,0
216029,CLASA,31596973,001105045005269530,KSTBC,2024-09-20,15,NO CONTACTADO,C1,0,PREDICTIVO,3,202409,15,13,0


In [122]:
df_clasa_test.shape

(1112769, 15)

In [16]:
def add_new_columns(df: pd.DataFrame) -> pd.DataFrame:
    df['INTENSIDAD_TOTAL'] = np.nan
    df['INTENSIDAD_DIRECTA'] = np.nan
    df['COBERTURA'] = np.nan
    df['CONTACTO_DIRECTO'] = np.nan
    df['TASA_CIERRE'] = np.nan
    
    return df

In [ ]:
df_rj_indicadores = df_clasa_test.groupby(['AGENCIA', 'FECHA_LLAMADA', 'MARCA', 'TRAMO', 'TIPO_MARCADOR', 'PRIORIDAD_TELEFONO']).agg({
    'INTENSIDAD_TOTAL': 'mean', 
    'INTENSIDAD_DIRECTA': lambda x: x[x != 0].mean(), 
    'COBERTURA': 'mean', 
    'CONTACTO_DIRECTO': lambda x: x.mean(skipna=True), 
    'TASA_CIERRE': lambda x: x.mean(skipna=True), 
    'CALIDAD_PROMESAS': lambda x: x.mean(skipna=True)
}).reset_index()

df_rj_indicadores.head()

In [ ]:
df_rj_indicadores = df_clasa_test.groupby(['AGENCIA', 'MARCA', 'FECHA_LLAMADA', 'TIPO_CONTACTO', 'TRAMO', 'PDP', 'TIPO_MARCADOR', 'PRIORIDAD_TELEFONO']).agg({
    'INTENSIDAD_TOTAL': 'mean', 
    'INTENSIDAD_DIRECTA': lambda x: x[x != 0].mean(), 
    'COBERTURA': 'mean', 
    'CONTACTABILIDAD_DIRECTA': lambda x: x.mean(skipna=True), 
    'TASA_CIERRE': lambda x: x.mean(skipna=True), 
    'CALIDAD_PROMESAS': lambda x: x.mean(skipna=True)
}).reset_index()

df_rj_indicadores['INTENSIDAD_TOTAL'] = df_rj_indicadores['INTENSIDAD_TOTAL'].round(2)
df_rj_indicadores['INTENSIDAD_DIRECTA'] = df_rj_indicadores['INTENSIDAD_DIRECTA'].round(2)
df_rj_indicadores['COBERTURA'] = df_rj_indicadores['COBERTURA'].round(4)
df_rj_indicadores['CONTACTABILIDAD_DIRECTA'] = df_rj_indicadores['CONTACTABILIDAD_DIRECTA'].round(2)
df_rj_indicadores['TASA_CIERRE'] = df_rj_indicadores['TASA_CIERRE'].round(2)
df_rj_indicadores['CALIDAD_PROMESAS'] = df_rj_indicadores['CALIDAD_PROMESAS'].round(2)

df_rj_indicadores.sort_values(by=['PERIODO', 'CARTERA', 'AGENCIA'], inplace=True)
df_rj_indicadores

In [ ]:
df_clasa_test['MARCA'].value_counts(dropna=False)

In [ ]:
df_clasa_test['TIPO_CONTACTO'].value_counts(dropna=False)

In [ ]:
df_clasa_test['TRAMO'].value_counts(dropna=False)

In [ ]:
df_clasa_test['PDP'].value_counts(dropna=False)

In [ ]:
df_clasa_test['TIPO_MARCADOR'].value_counts(dropna=False)